In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
%cd drive/My\ Drive/textaug/notebooks/Word2Vec

/content/drive/My Drive/textaug/notebooks/Word2Vec


In [3]:
!pip install --upgrade gensim -q
!pip install annoy -q

     |████████████████████████████████| 24.2MB 26.2MB/s 
     |████████████████████████████████| 645kB 2.8MB/s 


In [0]:
import numpy as np
import pandas as pd
import re
# from gensim.test.utils import datapath
# from gensim import utils
import gensim.models
from gensim.similarities.index import AnnoyIndexer
import random

In [0]:
class Word2VecAugmentation:
  def __init__(self, path_to_dictionary='mipt_vecs.w2v', indexer=None, cache_dict=False):
    #если нет словаря, то скачать его
    self.model = gensim.models.KeyedVectors.load_word2vec_format(path_to_dictionary, binary=True, unicode_errors='ignore')
    self.annoy_index = AnnoyIndexer(self.model, num_trees=10) if (indexer=='annoy') else None
    self.replace_dict = dict() if cache_dict else None

  def clear_replace_dict(self):
    if self.replace_dict is not None:
      self.replace_dict.clear()
    return

  def get_similar_words_dict(self, words_set):
    word_index = 0 if self.annoy_index is None else 1
    replace_dict = dict() if self.replace_dict is None else self.replace_dict
    for word in ( words_set - replace_dict.keys() ):
      if word in self.model:
        replace_dict[word] = self.model.most_similar(word, topn=word_index+1, indexer=self.annoy_index)[word_index][0]
    return replace_dict

  def augment_text(self, text, partition=0.5):
    words_set = set( re.findall(r'[а-яА-Яa-zA-Z]+', text) )
    replace_dict = self.get_similar_words_dict(words_set)
    return re.sub(r'[а-яА-Яa-zA-Z]+', lambda x: replace_dict[x.group()] if (random.random()<partition) and (x.group() in replace_dict) else x.group(), text)

чек

In [6]:
wv_aug = Word2VecAugmentation(cache_dict=False, indexer='annoy')

/usr/local/lib/python3.6/dist-packages/gensim/similarities/index.py:184: FutureWarning: The default argument for metric will be removed in future version of Annoy. Please pass metric='angular' explicitly.
  index = AnnoyIndex(num_features)


In [0]:
df1 = pd.read_csv('../../data/twitts/positive.csv', sep=';', header=None)[[3]].rename({3:'tweet'}, axis=1)
df2 = pd.read_csv('../../data/twitts/negative.csv', sep=';', header=None)[[3]].rename({3:'tweet'}, axis=1)

df = df1.append(df2, ignore_index=True)

In [0]:
import warnings  
warnings.filterwarnings(action='ignore',category=UserWarning,module='gensim')  
warnings.filterwarnings(action='ignore',category=FutureWarning,module='gensim')

In [9]:
%timeit wv_aug.augment_text(df.tweet[0], 1)
%timeit wv_aug.augment_text(df.tweet[0], 0.5)
%timeit wv_aug.augment_text(df.tweet[0], 0.1)

The slowest run took 19.55 times longer than the fastest. This could mean that an intermediate result is being cached.
1000 loops, best of 3: 1.21 ms per loop
1000 loops, best of 3: 1.21 ms per loop
1000 loops, best of 3: 1.21 ms per loop


In [10]:
%%time
t = df.tweet[:100].apply( lambda x: wv_aug.augment_text(x) )

CPU times: user 103 ms, sys: 2 ms, total: 105 ms
Wall time: 106 ms


In [11]:
%%time
t = df.tweet[:1000].apply( lambda x: wv_aug.augment_text(x) )

CPU times: user 818 ms, sys: 5 µs, total: 818 ms
Wall time: 822 ms


In [12]:
%%time
t = df.tweet[:10000].apply( lambda x: wv_aug.augment_text(x) )

CPU times: user 8.09 s, sys: 16.7 ms, total: 8.11 s
Wall time: 8.13 s


In [13]:
%%time
t = df.tweet.apply( lambda x: wv_aug.augment_text(x) )

CPU times: user 2min 58s, sys: 285 ms, total: 2min 59s
Wall time: 2min 59s


O(n)

In [16]:
n = random.randint(10, len(t))
for v in t[n-10:n]:
  print(v)

RT @Shlegel: Ахахаха :))) доброту утро RT @euromaidan: В Грузии парламентарии преподнесли в парламент белорусские флаги не­мало шарфы http://t.uk//MZZlpFu…
@SeniBellem впечатление… — же замечательно) во сколько дом была? И уже на локтях?)
@aiena_1996 хватает… боимся де­вушки дамочки вымарывать,тугодумные даже анонимные пишут что они хватает… нас женщины,оууу:D
RT @weduhycepiwyq: оставались такие романтичные, что над нашей переписки вконтакте плачущее поговаривали ЗБУ;)…! http://t.co/kjYN1YghX3
конечно понять найти человека, намекнувший полюбил приходило скажи несмотря на думаешь недостатки.  -Noize MC
И мне такого наконец-то нашла :))
RT @affects_1D: #HappyBirthdayIanFromRussia
секси, поздравляю с днем мистериальное:* http://t.co/CVhwEJd2i/ii
RT @ildrozdov: У "СР" субботу удалась,все)) “@107_0: В Мосгордуме сочли идиотизмом новацию сравнить ремонтную метро… году{ приветствовать Манделы //www://t.uk/…
Задай вопрос, попытается не жмот. :*  / //www://t.uk//R7acxP6aXG http://t.co/9QddSBvGy

теперь чек для кэша

In [0]:
wv_aug_cache = Word2VecAugmentation(cache_dict=True, indexer='annoy')

In [50]:
%%time
t = df.tweet[:100].apply( lambda x: wv_aug_cache.augment_text(x) )

CPU times: user 75.5 ms, sys: 23 µs, total: 75.5 ms
Wall time: 79.3 ms


In [52]:
%%time
t = df.tweet[:1000].apply( lambda x: wv_aug_cache.augment_text(x) )

CPU times: user 439 ms, sys: 2.96 ms, total: 442 ms
Wall time: 451 ms


In [0]:
wv_aug_cache.clear_replace_dict()

In [54]:
%%time
t = df.tweet[:10000].apply( lambda x: wv_aug_cache.augment_text(x) )

CPU times: user 5.25 s, sys: 3.96 ms, total: 5.26 s
Wall time: 5.27 s


In [0]:
wv_aug_cache.clear_replace_dict()

In [55]:
%%time
t = df.tweet[:50000].apply( lambda x: wv_aug_cache.augment_text(x) )

CPU times: user 49.8 s, sys: 9.88 ms, total: 49.8 s
Wall time: 49.9 s
